In [ ]:
# Check that the account has been saved properly
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService(name="qgss-2025")
service.saved_accounts()

{'qgss-2025': {'channel': 'ibm_quantum_platform',
  'url': 'https://cloud.ibm.com',
  'token': 'j4MxUf6EZ0xR8RxcjXJ1w6MIfv1u-NoxO3AXSBGRiTIR',
  'instance': 'crn:v1:bluemix:public:quantum-computing:us-east:a/f45e667ab0174f6d860e44a5b8aea0af:f81a572d-0e39-446e-b820-3ada6c1ac0aa::',
  'verify': True,
  'private_endpoint': False}}

In [ ]:
import rustworkx as rx
import numpy as np
import matplotlib.pyplot as plt
from rustworkx.visualization import mpl_draw as draw_graph
from qiskit_ibm_runtime import QiskitRuntimeService
from scipy.optimize import minimize

from qiskit import QuantumCircuit
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.quantum_info import SparsePauliOp, Statevector, DensityMatrix, Operator
from qiskit.circuit.library import QAOAAnsatz
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_histogram
from qiskit.transpiler import Layout

from qiskit_ibm_runtime import (
    Session,
    EstimatorV2 as Estimator,
    SamplerV2 as Sampler,
    EstimatorOptions,
)
from qiskit_ibm_runtime.debug_tools import Neat
from qiskit_aer import AerSimulator

from utils import zne_method, plot_zne, plot_backend_errors_and_counts
from qc_grader.challenges.qgss_2025 import (
    grade_lab2_ex1,
    grade_lab2_ex2,
    grade_lab2_ex3,
    grade_lab2_ex4,
    grade_lab2_ex5,
    grade_lab2_ex6a,
    grade_lab2_ex6b,
)

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

def find_best_metrics(backend) -> list[tuple[int or list, float]]:
    """Finds the best-performing qubits and qubit pair based on various hardware metrics."""
    
    properties = backend.properties()
    num_qubits = backend.num_qubits
    coupling_map = backend.coupling_map

    # Define metrics lists for the backend
    t1, t2, gate_error_x, readout_error, gate_error_ecr = [], [], [], [], []

    for i in range(num_qubits):
        t1.append(properties.t1(i))
        t2.append(properties.t2(i))
        gate_error_x.append(properties.gate_error(gate="x", qubits=i))
        readout_error.append(properties.readout_error(i))

    for pair in coupling_map:
        gate_error_ecr.append((pair, properties.gate_error(gate="ecr", qubits=pair)))

    # Find best qubit for each metric
    index_t1_max = t1.index(max(t1))
    max_t1 = t1[index_t1_max]

    index_t2_max = t2.index(max(t2))
    max_t2 = t2[index_t2_max]

    index_min_x_error = gate_error_x.index(min(gate_error_x))
    min_x_error = gate_error_x[index_min_x_error]

    index_min_readout = readout_error.index(min(readout_error))
    min_readout = readout_error[index_min_readout]

    min_ecr_pair, min_ecr_error = min(gate_error_ecr, key=lambda x: x[1])

    solutions = [
        [int(index_t1_max), max_t1],
        [int(index_t2_max), max_t2],
        [int(index_min_x_error), min_x_error],
        [int(index_min_readout), min_readout],
        [list(min_ecr_pair), min_ecr_error],
    ]
    return solutions

# Define the backend
service = QiskitRuntimeService(name="qgss-2025")
brisbane_backend = service.backend("ibm_brisbane")

# Submit the answer
grade_lab2_ex1(find_best_metrics)


Submitting your answer. Please wait...
Congratulations! 🎉 Your answer is correct.


In [ ]:
import rustworkx as rx
from qiskit.quantum_info import SparsePauliOp

def graph_to_Pauli(graph: rx.PyGraph) -> list[tuple[str, float]]:
    """Convert the graph to Pauli list."""
    pauli_list = []

    # For each edge, create a Pauli string with Z on the connected nodes
    for edge in graph.edge_list():
        z_string = ["I"] * len(graph)
        z_string[edge[0]] = "Z"
        z_string[edge[1]] = "Z"

        weight = graph.get_edge_data(edge[0], edge[1])
        pauli_term = ("".join(z_string)[::-1], weight)  # Reverse for little-endian format
        pauli_list.append(pauli_term)

    return pauli_list

# Submit your answer
grade_lab2_ex2(graph_to_Pauli)


Submitting your answer. Please wait...
Congratulations! 🎉 Your answer is correct.


In [ ]:
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler import CouplingMap

def accumulated_errors(backend, circuit):
    """Compute accumulated gate and readout errors for a given circuit on a specific backend."""

    # Initialize accumulators and counters
    acc_single_qubit_error = 0
    acc_two_qubit_error = 0
    single_qubit_gate_count = 0
    two_qubit_gate_count = 0
    acc_readout_error = 0

    # Get backend properties and configuration
    properties = backend.properties()
    config = backend.configuration()

    # Determine which two-qubit gate is used
    if "ecr" in config.basis_gates:
        two_qubit_gate = "ecr"
    elif "cz" in config.basis_gates:
        two_qubit_gate = "cz"
    elif "cx" in config.basis_gates:
        two_qubit_gate = "cx"
    else:
        raise ValueError("Unknown two-qubit gate")

    # Loop over circuit instructions
    for instr, qargs, _ in circuit.data:
        if instr.name in ["rz", "x", "sx"]:
            qubit = circuit.find_bit(qargs[0]).index
            try:
                gate_error = properties.gate_error(instr.name, [qubit])
                acc_single_qubit_error += gate_error
                single_qubit_gate_count += 1
            except:
                continue  # in case of missing calibration
        elif instr.name == two_qubit_gate:
            q0 = circuit.find_bit(qargs[0]).index
            q1 = circuit.find_bit(qargs[1]).index
            try:
                gate_error = properties.gate_error(instr.name, [q0, q1])
                acc_two_qubit_error += gate_error
                two_qubit_gate_count += 1
            except:
                continue

    # Readout errors: based on **measured** qubits
    measured_qubits = set()
    for instr, qargs, _ in circuit.data:
        if instr.name == "measure":
            measured_qubits.add(circuit.find_bit(qargs[0]).index)

    for qubit in measured_qubits:
        try:
            acc_readout_error += properties.readout_error(qubit)
        except:
            continue

    # Total error
    acc_total_error = acc_two_qubit_error + acc_single_qubit_error + acc_readout_error

    results = [
        acc_total_error,
        acc_two_qubit_error,
        acc_single_qubit_error,
        acc_readout_error,
        single_qubit_gate_count,
        two_qubit_gate_count,
    ]

    return results
grade_lab2_ex3(accumulated_errors)


Submitting your answer. Please wait...
Congratulations! 🎉 Your answer is correct.


In [ ]:
def find_paths_with_weight_sum_below_threshold(
    graph: rx.PyDiGraph,
    threshold: float,
    two_qubit_ops_list: list[int],
    logical_pair_list: list[list[int]],
) -> tuple[list[list[int]], list[float]]:
    """Find all valid paths through a graph whose weighted sum is below a given threshold."""
    valid_paths = []
    valid_weights = []

    # Iterate over all possible starting nodes in the graph
    for start_node in range(graph.num_nodes()):
        # Initialize the list of paths with a single-node path starting from the current node
        paths = [[start_node]]
        # Initialize the corresponding weights for each path (starting with 0)
        weights = [0]
        # Iterate through each step in the sequence of two-qubit operations
        for i in range(len(two_qubit_ops_list)):
            new_paths = []    
            new_weights = []  
            # Go through each current path and its weight
            for path, weight in zip(paths, weights):
                # Determine which node in the path we are going to expand from using logical_pair_list
                if logical_pair_list[i][0] < logical_pair_list[i][1]:
                    index_of_expanding_node = logical_pair_list[i][0]  # control qubit
                    node_to_expand_from = path[index_of_expanding_node]
                    # Explore all neighbors of the node_to_expand_from (directed)
                    for neighbor in graph.neighbors(node_to_expand_from):
                        if neighbor not in path and graph.has_edge(node_to_expand_from, neighbor):
                            edge_weight = two_qubit_ops_list[i] * graph.get_edge_data(node_to_expand_from, neighbor)
                            new_paths.append(path + [neighbor])
                            new_weights.append(weight + edge_weight)
                else:
                    index_of_expanding_node = logical_pair_list[i][1]  # target qubit
                    node_to_expand_from = path[index_of_expanding_node]
                    # Explore all undirected neighbors
                    for neighbor in graph.neighbors_undirected(node_to_expand_from):
                        if neighbor not in path and graph.has_edge(neighbor, node_to_expand_from):
                            edge_weight = two_qubit_ops_list[i] * graph.get_edge_data(neighbor, node_to_expand_from)
                            new_paths.append(path + [neighbor])
                            new_weights.append(weight + edge_weight)
            # Update paths and weights for next iteration
            paths = new_paths
            weights = new_weights

        # Filter paths with total weight below threshold
        for path, weight in zip(paths, weights):
            if weight < threshold:
                valid_paths.append(path)
                valid_weights.append(weight)

    return valid_paths, valid_weights
# Submit your answer using the following code
grade_lab2_ex4(find_paths_with_weight_sum_below_threshold)

Submitting your answer. Please wait...
Congratulations! 🎉 Your answer is correct.


In [ ]:
from utils import two_qubit_gate_errors_per_circuit_layout


In [ ]:
# Exercise 05 — Best Mapping: Full Solution for Seed Optimization

from qiskit import QuantumCircuit, transpile

# Try Aer first, fallback to BasicAer if Aer is unavailable
try:
    from qiskit_aer import Aer
    backend = Aer.get_backend('qasm_simulator')
except ImportError:
    from qiskit import BasicAer
    backend = BasicAer.get_backend('qasm_simulator')

# Import your gate error counting helper function
from utils import two_qubit_gate_errors_per_circuit_layout

# --- Prepare or load your target circuit ---
# Replace with your actual circuit if needed
qaoa_circuit = QuantumCircuit(3)
qaoa_circuit.h(0)
qaoa_circuit.cx(0, 1)
qaoa_circuit.cx(1, 2)
n = 3  # Set to the number of logical qubits in your circuit

# --- Define the seed optimization function ---
def finding_best_seed(circuit, backend):
    min_error = float('inf')
    best_circ = None
    best_seed = None
    best_gate_count = None
    for seed in range(500):
        transpiled = transpile(
            circuit,
            backend=backend,
            optimization_level=3,
            seed_transpiler=seed,
            layout_method="sabre"
        )
        err, gate_count = two_qubit_gate_errors_per_circuit_layout(transpiled, backend)
        if err < min_error:
            min_error = err
            best_circ = transpiled
            best_seed = seed
            best_gate_count = gate_count
    return best_circ, best_seed, min_error, best_gate_count

# --- Run the search for the best mapping ---
circuit_opt_seed_loop, best_seed_transpiler, min_err_acc_seed_loop, two_qubit_gate_count_seed_loop = finding_best_seed(qaoa_circuit, backend)

# --- Extract the best layout for the first n logical qubits ---
try:
    # For Qiskit 0.18+ transpiled circuit has .layout with .initial_layout and .get_physical_bits()
    best_layout = list(circuit_opt_seed_loop.layout.initial_layout.get_physical_bits().keys())[:n]
except Exception:
    # Fallback for older/other Qiskit versions, assume direct mapping
    best_layout = list(range(n))

print(f"Best transpiler seed: {best_seed_transpiler}")
print(f"Minimum accumulated two-qubit gate error: {min_err_acc_seed_loop:.3f}")
print(f"Two-qubit gate count for best seed: {two_qubit_gate_count_seed_loop}")
print(f"Best layout (first n logical qubits mapped to physical qubits):\n {best_layout}")

# --- Submit for grading (uncomment below if your environment supports it) ---
grade_lab2_ex5(finding_best_seed)


Best transpiler seed: 0
Minimum accumulated two-qubit gate error: 2.000
Two-qubit gate count for best seed: 2
Best layout (first n logical qubits mapped to physical qubits):
 [0, 1, 2]
